In [7]:

from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [8]:
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [9]:
extracted_data=load_pdf_file(data='Data/')

In [10]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

In [13]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [14]:
embeddings = download_hugging_face_embeddings()

C:\Users\tejay\AppData\Local\Temp\ipykernel_18140\4043855499.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [15]:
import os
os.environ["PINECONE_API_KEY"] = "pcsk_63wajS_6A1MQAAjTMqi9pBi4hVz4pYHKEGVnxP5qqqtGtRg5YgY39UsEwfWPgwQ5mvU19i"

In [16]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

# Set API Key and environment variables
os.environ["PINECONE_API_KEY"] = "pcsk_63wajS_6A1MQAAjTMqi9pBi4hVz4pYHKEGVnxP5qqqtGtRg5YgY39UsEwfWPgwQ5mvU19i"
os.environ["PINECONE_ENVIRONMENT"] = "us-east-1" # e.g., us-west-1

# Initialize Pinecone
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

# Connect to your index
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name="medibuddy",
    embedding=embeddings
)


KeyboardInterrupt: 

In [17]:
from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name="medibuddy",
    embedding=embeddings
)

In [18]:
docsearch

In [19]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":20})

In [20]:
retrieved_docs = retriever.invoke("What is  Acrocyanosis?")

In [ ]:
retrieved_docs

[Document(id='4f029945-be6b-48d2-9a81-8830bbeec856', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 44.0, 'page_label': '45', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\gale.pdf', 'total_pages': 637.0}, page_content='Acquired hypogammaglobulinemia see\nCommon variable immunodeficiency\nAcquired immunodeficiency syndrome see\nAIDS\nAcrocyanosis\nDefinition\nAcrocyanosis is a decrease in the amount of oxygen\ndelivered to the extremities. The hands and feet turn blue\nbecause of the lack of oxygen. Decreased blood supply\nto the affected areas is caused by constriction or spasm of\nsmall blood vessels.\nDescription\nAcrocyanosis is a painless disorder caused by constric-'),
 Document(id='e6827ad6-96db-4365-ac5f-1eb9ddde29e6', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 44.0, 'page_label': '45', 'producer': 'PDFlib+PDI 5.0

In [21]:
# Extract only the text from retrieved documents
retrieved_texts = [doc.page_content for doc in retrieved_docs]

for text in retrieved_texts:
    print(text)


Acquired hypogammaglobulinemia see
Common variable immunodeficiency
Acquired immunodeficiency syndrome see
AIDS
Acrocyanosis
Definition
Acrocyanosis is a decrease in the amount of oxygen
delivered to the extremities. The hands and feet turn blue
because of the lack of oxygen. Decreased blood supply
to the affected areas is caused by constriction or spasm of
small blood vessels.
Description
Acrocyanosis is a painless disorder caused by constric-
Acquired hypogammaglobulinemia see
Common variable immunodeficiency
Acquired immunodeficiency syndrome see
AIDS
Acrocyanosis
Definition
Acrocyanosis is a decrease in the amount of oxygen
delivered to the extremities. The hands and feet turn blue
because of the lack of oxygen. Decreased blood supply
to the affected areas is caused by constriction or spasm of
small blood vessels.
Description
Acrocyanosis is a painless disorder caused by constric-
Acquired hypogammaglobulinemia see
Common variable immunodeficiency
Acquired immunodeficiency syndrome

In [ ]:
from flask import Flask, request, jsonify
from langchain_pinecone import PineconeVectorStore
import os
os.environ["PINECONE_API_KEY"] = "pcsk_63wajS_6A1MQAAjTMqi9pBi4hVz4pYHKEGVnxP5qqqtGtRg5YgY39UsEwfWPgwQ5mvU19i"


def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

app = Flask(__name__)

embeddings = download_hugging_face_embeddings()

# Initialize Pinecone Vector Store
docsearch = PineconeVectorStore.from_existing_index(
    index_name="medibuddy",
    embedding=embeddings
)
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 6})

@app.route('/query', methods=['POST'])
def query():
    data = request.get_json()
    query_text = data.get("query")
    
    if not query_text:
        return jsonify({"error": "Query text is required"}), 400

    retrieved_docs = retriever.invoke(query_text)
    retrieved_texts = [doc.page_content for doc in retrieved_docs]

    return jsonify({"texts": retrieved_texts})

if __name__ == '__main__':
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Mar/2025 12:11:37] "POST /query HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2025 12:12:04] "POST /query HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2025 12:12:54] "POST /query HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2025 12:13:37] "POST /query HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2025 12:17:56] "POST /query HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2025 12:18:40] "POST /query HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2025 12:19:14] "POST /query HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2025 12:22:17] "POST /query HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2025 12:55:54] "POST /query HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2025 12:57:13] "POST /query HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2025 12:57:54] "POST /query HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2025 12:58:47] "POST /query HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2025 12:59:26] "POST /query HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2025 12:59:55] "POST /query HTTP/1.1" 200 -


In [5]:
from flask import Flask, request, jsonify
from langchain_pinecone import PineconeVectorStore
from sentence_transformers import SentenceTransformer

app = Flask(__name__)

# Load embeddings using Hugging Face model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Initialize Pinecone Vector Store
docsearch = PineconeVectorStore.from_existing_index(
    index_name="medibuddy",
    embedding=model
)
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 6})

@app.route('/query', methods=['POST'])
def query():
    data = request.get_json()
    query_text = data.get("query")
    
    if not query_text:
        return jsonify({"error": "Query text is required"}), 400

    retrieved_docs = retriever.invoke(query_text)
    retrieved_texts = [doc.page_content for doc in retrieved_docs]

    return jsonify({"texts": retrieved_texts})

if __name__ == '__main__':
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
[2025-03-26 12:02:03,266] ERROR in app: Exception on /query [POST]
Traceback (most recent call last):
  File "c:\Users\tejay\anaconda3\envs\rag\lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\tejay\anaconda3\envs\rag\lib\site-packages\flask\app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\tejay\anaconda3\envs\rag\lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\tejay\anaconda3\envs\rag\lib\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "C:\Users\tejay\AppData\Local\Temp\ipykernel_18140\2430652593.py", line 25, in query
    retrieved_docs = retriever.invoke(query_text)
  File "c:\Users\tejay\anaconda3\envs\rag\lib\site-p

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Mar/2025 12:04:49] "POST /query HTTP/1.1" 500 -
127.0.0.1 - - [26/Mar/2025 12:05:08] "POST /query HTTP/1.1" 500 -
